### Loss_ASoftmax
이 함수는 softmax 손실 함수를 수정하여 분류 모델에서의 효과를 높이기 위해 사용됩니다.
함수는 입력으로 특징 벡터(x), 레이블(y), 스케일 하이퍼파라미터(l), 클래스 수(num_cls), 마진(m)과 이름(name)을 받습니다. 이 함수에서는 가중치 행렬(w)을 선언하고, A-softmax 손실을 계산합니다.
#### 계산
가중치 행렬과 특징 벡터를 곱하여 로짓(logits)을 계산합니다.
로짓을 특정 기준으로 변환합니다. 여기에서는 A-softmax에서 제안된 방법을 사용합니다. 이 방법은 로짓 벡터를 단위 벡터로 변환하여 각 클래스와의 거리를 측정하고, 이 거리에 스케일 하이퍼파라미터(l)을 곱합니다. 그리고 거리를 각도(cosine)로 변환합니다. 이를 통해 로짓 벡터의 스케일이 잘 조정됩니다.
변환된 로짓을 사용하여 손실을 계산합니다. 여기에서는 A-softmax의 변형 중 하나인 additive margin softmax(AM-softmax)를 사용합니다. 이 방법은 기존의 소프트맥스 함수와는 달리, 로짓과 레이블 사이의 마진(m)을 추가하여 손실을 계산합니다.
이렇게 계산된 A-softmax 손실은 클래스 간의 거리를 더 잘 보존하는 특징을 가지고 있어, 분류 모델의 성능 향상에 기여합니다.

그림에서 얼굴이 있는 영역을 알아낸다. (face location)
얼굴 영역에서 눈, 코, 입 등 68개의 주요 좌표를 추출한다. (facial landmarks 추출)
68개의 좌표를 128개의 숫자로 변환한다. (face encoding) 예시임)
xw는 입력 x와 가중치 행렬 w의 곱으로 계산된 점수이며, w는 각 클래스에 해당하는 점수를 계산하는 가중치를 나타냅니다. logits 변수는 A-softmax 손실 함수에서 출력으로 사용됩니다.

In [ ]:
#다시 시행하기 전에 코드실행
tf.reset_default_graph()

In [ ]:
import keras
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import cv2
import os
from matlab_cp2tform import get_similarity_transform_for_cv2
import matplotlib.pyplot as plt
import matplotlib.pylab as plt1
from mtcnn.mtcnn import MTCNN
detector = MTCNN()
import math
from random import shuffle
from sklearn.preprocessing import OneHotEncoder
import numpy as np   
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
losslist=[]
acclist=[]
epochlist=[]
elist = []

def prelu(_x):
    alphas = tf.get_variable('alpha', _x.get_shape()[-1],
                       initializer=tf.constant_initializer(0.0),
                        dtype=tf.float32)
    pos = tf.nn.relu(_x)
    neg = alphas * (_x - abs(_x)) * 0.5
    return pos+neg
    
def alignment(src_img,src_pts):   #얼굴 정렬
    ref_pts = [ [30.2946, 51.6963],[65.5318, 51.5014],
        [48.0252, 71.7366],[33.5493, 92.3655],[62.7299, 92.2041] ]
    crop_size = (96, 112)
    src_pts = np.array(src_pts).reshape(5,2)

    s = np.array(src_pts).astype(np.float32)
    r = np.array(ref_pts).astype(np.float32)

    tfm = get_similarity_transform_for_cv2(s, r)
    face_img = cv2.warpAffine(src_img, tfm, crop_size)
    return face_img


def get_batch(step,bz):
    shuffle(idx)
    x=flist[idx[step*bz:(step+1)*bz]]
    y=label[idx[step*bz:(step+1)*bz]]
    resultx=[]
    for ix in x:
        img=plt.imread(ix[0])
        tmp = detector.detect_faces(img)[0]['keypoints']
        landmark = []
        for t in tmp.items(): #딕셔너리 키&값 얻기
            for tt in t[1]: #딕셔너리 값 얻기->left eye위치, right eye위치....
                landmark.append(tt) #얼굴 값들 landmark에 저장.
        img=alignment(img,landmark) #자른 이미지
        resultx.append(img) #결과
    resultx=np.array(resultx)
    y=y.reshape(bz,)
    return resultx, y

datapath='C:/Users/Administrator/Desktop/두희 얼굴인식/face_recognize code/face'
flist=[]
for(dirpath,dirnames,filenames) in os.walk(datapath):
    for filename in filenames:
        if 'CASIA-WebFace' in dirpath:continue
        flist.append(dirpath+'/'+filename)
label=[]
for f in flist[:10]:
    label.append(f.split('/')[-1])
classnum=10
labelset = list(set(label))
for i in range(len(label)):
    label[i] = labelset.index(label[i])

# sess = tf.Session()
# label = sess.run(tf.one_hot(label, classnum))
label=np.array(label)

label=label.reshape(-1,1)
flist = np.array(flist[:10])
flist = flist.reshape(-1, 1)
idx=[i for i in range(10)]
shuffle(idx)

def Loss_ASoftmax(x, y, l, num_cls, m = 4, name = 'asoftmax'):

    xs = x.get_shape()
    w = tf.get_variable("asoftmax/W", [xs[1], num_cls], dtype=tf.float32,
            initializer=tf.glorot_uniform_initializer())

    eps = 1e-8

    xw = tf.matmul(x,w)

    if m == 0:
        return xw, tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=xw))

    w_norm = tf.norm(w, axis = 0) + eps
    logits = xw/w_norm

    if y is None:
        return logits, None

    ordinal = tf.constant(list(range(0, xs[0])), tf.int64)
    ordinal_y = tf.stack([ordinal, y], axis = 1)

    x_norm = tf.norm(x, axis = 1) + eps

    sel_logits = tf.gather_nd(logits, ordinal_y)

    cos_th = tf.div(sel_logits, x_norm)

    if m == 1:

        loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits
                              (labels=y,  logits=logits))

    else:

        if m == 2:

            cos_sign = tf.sign(cos_th)
            res = 2*tf.multiply(cos_sign, tf.square(cos_th)) - 1

        elif m == 4:

            cos_th2 = tf.square(cos_th)
            cos_th4 = tf.pow(cos_th, 4)
            sign0 = tf.sign(cos_th)
            sign3 = tf.multiply(tf.sign(2*cos_th2 - 1), sign0)
            sign4 = 2*sign0 + sign3 - 3
            res = sign3*(8*cos_th4 - 8*cos_th2 + 1) + sign4
        else:
            raise ValueError('unsupported value of m')

        scaled_logits = tf.multiply(res, x_norm)

        f = 1.0/(1.0+l)
        ff = 1.0 - f
        comb_logits_diff = tf.add(logits, tf.scatter_nd(ordinal_y, tf.subtract(scaled_logits, sel_logits), logits.get_shape()))
        updated_logits = ff*logits + f*comb_logits_diff

        loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=updated_logits))
    return logits, loss

class SphereFace():
    def __init__(self,bz,m=4):
        self.class_num=classnum
        self.m=m
        self.lr=0.01
        self.batch=bz
        self.x=tf.placeholder(dtype=tf.float32,shape=[self.batch,112,96,3])#[b,112,96,3]
        self.y=tf.placeholder(dtype=tf.int64,shape=[self.batch,])
        #self.x_t=tf.placeholder(dtype=tf.float32,shape=[10,112,96,3])#[b,112,96,3]
        #self.y_t=tf.placeholder(dtype=tf.int64,shape=[10,])
        
        self.conv1_1=self.add_conv(filter_shape=[3,3,3,64],
                                 bias_shape=[64],
                                 input=self.x,strides=2,)#[b,56,48,64]
      
        with tf.variable_scope('relu1_1'):
            self.relu1_1=prelu(self.conv1_1)
        self.result=self.relu1_1
        self.conv1_2=self.add_conv(filter_shape=[3,3,64,64],input=self.result,
                                   bias_shape=[64])
        
        with tf.variable_scope('relu1_2'):
            self.relu1_2=prelu(self.conv1_2)
        self.conv1_3=self.add_conv(filter_shape=[3,3,64,64],input=self.relu1_2,
                                   bias_shape=[64])
        with tf.variable_scope('relu1_3'):
            self.relu1_3=prelu(self.conv1_3)
        self.result+=self.relu1_3

        self.conv2_1=self.add_conv(filter_shape=[3,3,64,128]
                                   ,bias_shape=[128],input=self.result,strides=2)#[b,28,24,128]
        with tf.variable_scope('relu2_1'):
            self.relu2_1=prelu(self.conv2_1)
        self.result = self.relu2_1
        self.conv2_2=self.add_conv(filter_shape=[3,3,128,128],bias_shape=[128],input=self.relu2_1)
        with tf.variable_scope('relu2_2'):
            self.relu2_2=prelu(self.conv2_2)
        self.conv2_3 = self.add_conv(filter_shape=[3,3,128,128],bias_shape=[128],input=self.relu2_2)
        with tf.variable_scope('relu2_3'):
            self.relu2_3 = prelu(self.conv2_3)
        self.result+=self.relu2_3

        self.conv2_4 = self.add_conv(filter_shape=[3,3,128,128],bias_shape=[128],input=self.result)
        with tf.variable_scope('relu2_4'):
            self.relu2_4 = prelu(self.conv2_4)
        self.conv2_5 = self.add_conv(filter_shape=[3,3,128,128],bias_shape=[128],input=self.relu2_4)
        with tf.variable_scope('relu2_5'):
            self.relu2_5 = prelu(self.conv2_5)
        self.result+=self.relu2_5


        self.conv3_1 = self.add_conv(filter_shape=[3,3,128,256],bias_shape=[256]
                                     ,input=self.result,strides=2) #[b,14,12,256]
        with tf.variable_scope('relu3_1'):
            self.relu3_1 = prelu(self.conv3_1)
        self.result = self.relu3_1
        self.conv3_2 = self.add_conv(filter_shape=[3,3,256,256],bias_shape=[256],input=self.relu3_1)
        with tf.variable_scope('relu3_2'):
            self.relu3_2 = prelu(self.conv3_2)
        self.conv3_3 = self.add_conv([3,3,256,256],[256],self.relu3_2)
        with tf.variable_scope('relu3_3'):
            self.relu3_3 = prelu(self.conv3_3)
        self.result+=self.relu3_3

        self.conv3_4 = self.add_conv([3,3,256,256],[256],self.result)
        with tf.variable_scope('relu3_4'):
            self.relu3_4 = prelu(self.conv3_4)
        self.conv3_5 = self.add_conv([3,3,256,256],[256],self.relu3_4)
        with tf.variable_scope('relu3_5'):
            self.relu3_5 = prelu(self.conv3_5)
        self.result+=self.relu3_5

        self.conv3_6 = self.add_conv([3,3,256,256],[256],self.result)
        with tf.variable_scope('relu3_6'):
            self.relu3_6 = prelu(self.conv3_6)
        self.conv3_7 = self.add_conv([3,3,256,256],[256],self.relu3_6)
        with tf.variable_scope('relu3_7'):
            self.relu3_7 = prelu(self.conv3_7)
        self.result+=self.relu3_7

        self.conv3_8 = self.add_conv([3,3,256,256],[256],self.result)
        with tf.variable_scope('relu3_8'):
            self.relu3_8 = prelu(self.conv3_8)
        self.conv3_9 = self.add_conv([3,3,256,256],[256],self.relu3_8)
        with tf.variable_scope('relu3_9'):
            self.relu3_9 =prelu(self.conv3_9)
        self.result+=self.relu3_9

        self.conv4_1 = self.add_conv([3,3,256,512],[512],self.result,strides=2) #[b,7,6,512]
        with tf.variable_scope('relu4_1'):
            self.relu4_1 =prelu(self.conv4_1)
        self.result = self.relu4_1
        self.conv4_2 = self.add_conv([3,3,512,512],[512],self.relu4_1)
        with tf.variable_scope('relu4_2'):
            self.relu4_2 =prelu(self.conv4_2)
        self.conv4_3 = self.add_conv([3,3,512,512],[512],self.relu4_2)
        with tf.variable_scope('relu4_3'):
            self.relu4_3 = prelu(self.conv4_3)
        self.result+=self.relu4_3

        self.poutput=self.add_dense(self.result)
        
        self.output,self.loss=Loss_ASoftmax(self.poutput,self.y,l=1.0,num_cls=self.class_num,m=4)
        self.train_step=tf.train.AdamOptimizer(1e-4).minimize(self.loss)
        self.corrects=tf.equal(tf.argmax(self.output,1),self.y) 
        self.acc=tf.reduce_mean(tf.cast(self.corrects,tf.float32))

        self.init=tf.global_variables_initializer()
        self.sess=tf.Session()
        self.sess.run(self.init)
        for i in range(300):
            x, y = get_batch(step=0, bz=10)  # 학습 데이터셋에서 무작위로 샘플링한 100개의 데이터로 구성된 'batch'를 가져옴
            #x = x/255.0             #정규화
            #x, x_test, y, y_test = train_test_split(x,y,test_size=0.3)
            self.sess.run(self.train_step, feed_dict={self.x: x, self.y: y})    # placeholder x, y_에 샘플링된 batch_xs, batch_ys를 공급함     
        loss= self.sess.run(self.acc, feed_dict={self.x: x, self.y:y})
        acc = self.sess.run(self.loss, feed_dict={self.x: x, self.y:y})
        print("Accuracy",acc)
        print("Loss", loss)
        losslist.append(loss)
        acclist.append(acc)
        #print("test accuracy", self.sess.run(self.acc,feed_dict={self.x_t: x_test, self.y_t: y_test}))
        
    def add_dense(self,input):
        input=tf.reshape(input,[-1,7*6*512])
        output=tf.layers.dense(inputs=input,units=512)
        return output

    def weight_variable(self,shape):
        init=tf.truncated_normal(shape=shape,stddev=0.01)
        return tf.Variable(init)

    def add_conv(self,filter_shape,bias_shape,input,strides=1,padding='SAME'):
        w1=self.weight_variable(filter_shape)
        b1=self.weight_variable(bias_shape)
        return tf.nn.conv2d(input,w1,strides=[1,strides,strides,1],padding=padding)+b1

from matplotlib.patches import Rectangle
if __name__=='__main__':
    bz=10
    model=SphereFace(bz)


Instructions for updating:
non-resource variables are not supported in the long term
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


C:\Users\Administrator\AppData\Local\Temp\ipykernel_10460\2992277142.py:273: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  output=tf.layers.dense(inputs=input,units=512)
C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


In [ ]:
np.save('a_loss.npy',losslist)
np.save('a_acc.npy', acclist)
plt1.xlabel('epoch')
plt1.ylabel('loss&acc')
plt_dict={}
plt_dict['loss']=losslist
plt_dict['acc']=acclist
plt_color_array=['blue','gree']
proxy=[]
legend_array=[]
for index,(tmp,lora)in enumerate(plt_dict.items()):
    color=plt_color_array[index]
    plt1.plot(range(len(epochlist)),epochlist,'-%s'%color[0])
    proxy.append(Rectangle((0,0),0,0,facecolor=color))
    legend_array.append(tmp)
plt1.legend(proxy,legend_array)
plt1.show()